# **Welcome to the MRInference machine learning tutorial!**
by Sandra Vieira

---
This webpage contains a brief step-by-step tutorial on the implementation of a standard supervised machine learning pipeline using Python programming language. Before this tutorial make sure to go through the pre-recorded lectures:  

*   Introduction to Machine Learning
*   The Machine Learning Pipeline 


---
##Machine Learning: Methods and Applications to Brain Disorders
This tutorial and both pre-recorded lectures above are based on the book [Machine Learning: Methods and Applications to Brain Disorders](https://www.amazon.co.uk/Machine-Learning-Methods-Applications-Disorders/dp/0128157399). The pre-recorded lectures are based on chapters 1-3 and this tutorial is a shorter version of Chapter 19. You can access the full tutorial of Chapter 19 [here](https://github.com/MLMH-Lab/How-To-Build-A-Machine-Learning-Model).

---  
## Aim and structure of the tutorial
For this tutorial you will use a toy dataset containing the grey matter volume and thickness from different brain regions extracted with FreeSurfer to classify patients with schizophrenia and healthy controls using a Support Vector Machine (SVM). The main steps of the tutrial will follow the pipeline presented in lecture The Machine Learning Pipeline and are shown in the figure below.

![workflow](https://raw.githubusercontent.com/MLMH-Lab/How-To-Build-A-Machine-Learning-Model/master/figures/Figure%201.png)



## Importing libraries

Python language is organised in libraries. Each library contains a set of functions for a specific purpose. For example, numpy is a popular library for manipulating numerical data, while pandas is most commonly used to handle tabular data. There are several libraries for machine learning analysis; in this tutorial we will use scikitlearn. 

In [ ]:
# SNIPPET 1

# Manipulate data
import numpy as np
import pandas as pd

# Plots
import seaborn as sns
import matplotlib.pyplot as plt

# Statistical tests
import scipy.stats as stats

# Machine learning
from sklearn.svm import LinearSVC
from sklearn.externals import joblib
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV

# Ignore WARNING
import warnings

warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Some steps in our analysis will be subjected to randomness.  We should set the  seed value to a fixed number to guarantee that we get the same results every time we run the code. 

In [ ]:
# SNIPPET 2
random_seed = 1
np.random.seed(random_seed)

## Problem formulation

 In this tutorial, our machine learning problem is: 

> *Classify patients with schizophrenia and healthy controls using structural MRI data.*

From this formulation we can derive the main elements of our machine learning problem:

*   **Features**: Structural MRI data
*   **Task**: Binary classification
*   **Target**: Patients with schizophrenia and healthy controls



---



## Data Preparation

The aim of this step is to perform a series of statistical analyses to get the data ready for the machine learning model. In this tutorial, we will assume the data is ready to be analysed. However, in a real project we would want to pay close attention to several things including class imbalance (N HC vs N SZ), missing data (data imputation?), confounding variables (age, sex?), dimensionality (N features vs N participants).

### Loading the data

In [ ]:
# SNIPPET 4
# dataset_file = Path('./Chapter_19_data.csv')
dataste_url = 'https://raw.githubusercontent.com/sandramv/MRInference_ML_Tutorial/main/ml_tutorial_data.csv'
dataset_df = pd.read_csv(dataste_url, index_col='ID')

In [ ]:
# SNIPPET 6
dataset_df[0:6]

,Diagnosis,Left Lateral Ventricle,Left Inf Lat Vent,Left Cerebellum White Matter,Left Cerebellum Cortex,Left Thalamus Proper,Left Caudate,Left Putamen,Left Pallidum,rd Ventricle,th Ventricle,Brain Stem,Left Hippocampus,Left Amygdala,CSF,Left Accumbens area,Left VentralDC,Right Lateral Ventricle,Right Inf Lat Vent,Right Cerebellum White Matter,Right Cerebellum Cortex,Right Thalamus Proper,Right Caudate,Right Putamen,Right Pallidum,Right Hippocampus,Right Amygdala,Right Accumbens area,Right VentralDC,CC Posterior,CC Mid Posterior,CC Central,CC Mid Anterior,CC Anterior,lh bankssts volume,lh caudalanteriorcingulate volume,lh caudalmiddlefrontal volume,lh cuneus volume,lh entorhinal volume,lh fusiform volume,...,lh superiortemporal thickness,lh supramarginal thickness,lh frontalpole thickness,lh temporalpole thickness,lh transversetemporal thickness,lh insula thickness,rh bankssts thickness,rh caudalanteriorcingulate thickness,rh caudalmiddlefrontal thickness,rh cuneus thickness,rh entorhinal thickness,rh fusiform thickness,rh inferiorparietal thickness,rh inferiortemporal thickness,rh isthmuscingulate thickness,rh lateraloccipital thickness,rh lateralorbitofrontal thickness,rh lingual thickness,rh medialorbitofrontal thickness,rh middletemporal thickness,rh parahippocampal thickness,rh paracentral thickness,rh parsopercularis thickness,rh parsorbitalis thickness,rh parstriangularis thickness,rh pericalcarine thickness,rh postcentral thickness,rh posteriorcingulate thickness,rh precentral thickness,rh precuneus thickness,rh rostralanteriorcingulate thickness,rh rostralmiddlefrontal thickness,rh superiorfrontal thickness,rh superiorparietal thickness,rh superiortemporal thickness,rh supramarginal thickness,rh frontalpole thickness,rh temporalpole thickness,rh transversetemporal thickness,rh insula thickness
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
c001,hc,4226.907844,414.407845,12242.90784,43410.50784,7020.107844,4133.407844,6467.707844,2048.207844,825.507845,1751.707844,18918.50784,3423.907844,917.007845,907.207845,691.607845,3500.107844,3517.007844,568.407845,13079.90784,44261.50784,6855.307844,4248.407844,6746.707844,1941.307844,3427.807844,1297.107844,618.107845,3837.307844,812.507845,429.107845,520.707845,407.907845,745.407845,1242.007844,911.007845,4082.007844,3236.007844,1992.007844,8908.007844,...,2.464844,2.409844,2.712844,1.940844,2.206844,2.895844,2.320844,2.229844,2.517844,1.879844,3.004844,2.726844,2.516844,2.604844,2.430844,2.323844,2.507844,2.072844,2.571844,2.731844,2.866844,2.279844,2.505844,2.828844,2.433844,1.523844,1.999844,2.487844,2.484844,2.312844,2.440844,2.522844,2.656844,2.123844,2.638844,2.420844,2.489844,2.235844,2.300844,2.645844
c002,hc,4954.912699,414.812699,16519.51270,38808.31270,7013.312699,3882.912699,5781.012699,1735.912699,457.512699,1123.312699,20193.91270,3582.712699,1578.712699,708.212699,593.812699,3802.312699,3420.612699,258.512699,16028.81270,44035.41270,6654.512699,3477.012699,5121.112699,1619.612699,3322.212699,1402.112699,529.412699,3842.312699,1034.512699,446.112699,495.012699,772.512699,815.412699,2596.012699,1493.012699,7759.012699,3268.012699,1850.012699,9055.012699,...,2.635699,2.560699,3.013699,2.452699,2.308699,2.859699,2.573699,2.141699,2.687699,1.904699,3.248699,2.408699,2.485699,2.558699,2.233699,2.181699,2.595699,2.095699,2.375699,2.659699,2.301699,2.456699,2.426699,2.823699,2.513699,1.967699,2.002699,2.297699,2.625699,2.273699,2.507699,2.470699,2.645699,2.132699,2.848699,2.425699,2.883699,2.622699,2.322699,2.673699
c003,hc,4470.611989,370.111989,10193.51199,38637.51199,5802.911989,2941.711989,5802.511989,1467.411989,835.011989,1050.011989,17577.51199,3338.211989,1318.311989,754.911989,702.611989,3444.511989,4097.511989,157.611989,14706.71199,42082.01199,5799.311989,3225.411989,4863.311989,1402.311989,3645.711989,1347.911989,588.011989,3924.011989,1067.911989,450.011989,492.411989,476.011989,888.611989,2556.011989,1633.011989,6815.011

In [ ]:
# SNIPPET 8
print('Number of features = %d' % dataset_df.shape[1])
print('Number of participants = %d' % dataset_df.shape[0])

Number of features = 170
Number of participants = 740


In [ ]:
# SNIPPET 11
dataset_df['Diagnosis'].value_counts()

sz    372
hc    368
Name: Diagnosis, dtype: int64

### Feature set and target

Our next step is to retrieve the target and features from the dataset.

In [ ]:
# SNIPPET 17
# Target
targets_df = dataset_df['Diagnosis']

# Features
features_names = dataset_df.columns[1:]
features_df = dataset_df[features_names]

In [ ]:
# SNIPPET 17a
features_df

,Left Lateral Ventricle,Left Inf Lat Vent,Left Cerebellum White Matter,Left Cerebellum Cortex,Left Thalamus Proper,Left Caudate,Left Putamen,Left Pallidum,rd Ventricle,th Ventricle,Brain Stem,Left Hippocampus,Left Amygdala,CSF,Left Accumbens area,Left VentralDC,Right Lateral Ventricle,Right Inf Lat Vent,Right Cerebellum White Matter,Right Cerebellum Cortex,Right Thalamus Proper,Right Caudate,Right Putamen,Right Pallidum,Right Hippocampus,Right Amygdala,Right Accumbens area,Right VentralDC,CC Posterior,CC Mid Posterior,CC Central,CC Mid Anterior,CC Anterior,lh bankssts volume,lh caudalanteriorcingulate volume,lh caudalmiddlefrontal volume,lh cuneus volume,lh entorhinal volume,lh fusiform volume,lh inferiorparietal volume,...,lh superiortemporal thickness,lh supramarginal thickness,lh frontalpole thickness,lh temporalpole thickness,lh transversetemporal thickness,lh insula thickness,rh bankssts thickness,rh caudalanteriorcingulate thickness,rh caudalmiddlefrontal thickness,rh cuneus thickness,rh entorhinal thickness,rh fusiform thickness,rh inferiorparietal thickness,rh inferiortemporal thickness,rh isthmuscingulate thickness,rh lateraloccipital thickness,rh lateralorbitofrontal thickness,rh lingual thickness,rh medialorbitofrontal thickness,rh middletemporal thickness,rh parahippocampal thickness,rh paracentral thickness,rh parsopercularis thickness,rh parsorbitalis thickness,rh parstriangularis thickness,rh pericalcarine thickness,rh postcentral thickness,rh posteriorcingulate thickness,rh precentral thickness,rh precuneus thickness,rh rostralanteriorcingulate thickness,rh rostralmiddlefrontal thickness,rh superiorfrontal thickness,rh superiorparietal thickness,rh superiortemporal thickness,rh supramarginal thickness,rh frontalpole thickness,rh temporalpole thickness,rh transversetemporal thickness,rh insula thickness
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
c001,4226.907844,414.407845,12242.907840,43410.50784,7020.107844,4133.407844,6467.707844,2048.207844,825.507845,1751.707844,18918.50784,3423.907844,917.007845,907.207845,691.607845,3500.107844,3517.007844,568.407845,13079.90784,44261.50784,6855.307844,4248.407844,6746.707844,1941.307844,3427.807844,1297.107844,618.107845,3837.307844,812.507845,429.107845,520.707845,407.907845,745.407845,1242.007844,911.007845,4082.007844,3236.007844,1992.007844,8908.007844,12371.007840,...,2.464844,2.409844,2.712844,1.940844,2.206844,2.895844,2.320844,2.229844,2.517844,1.879844,3.004844,2.726844,2.516844,2.604844,2.430844,2.323844,2.507844,2.072844,2.571844,2.731844,2.866844,2.279844,2.505844,2.828844,2.433844,1.523844,1.999844,2.487844,2.484844,2.312844,2.440844,2.522844,2.656844,2.123844,2.638844,2.420844,2.489844,2.235844,2.300844,2.645844
c002,4954.912699,414.812699,16519.512700,38808.31270,7013.312699,3882.912699,5781.012699,1735.912699,457.512699,1123.312699,20193.91270,3582.712699,1578.712699,708.212699,593.812699,3802.312699,3420.612699,258.512699,16028.81270,44035.41270,6654.512699,3477.012699,5121.112699,1619.612699,3322.212699,1402.112699,529.412699,3842.312699,1034.512699,446.112699,495.012699,772.512699,815.412699,2596.012699,1493.012699,7759.012699,3268.012699,1850.012699,9055.012699,12232.012700,...,2.635699,2.560699,3.013699,2.452699,2.308699,2.859699,2.573699,2.141699,2.687699,1.904699,3.248699,2.408699,2.485699,2.558699,2.233699,2.181699,2.595699,2.095699,2.375699,2.659699,2.301699,2.456699,2.426699,2.823699,2.513699,1.967699,2.002699,2.297699,2.625699,2.273699,2.507699,2.470699,2.645699,2.132699,2.848699,2.425699,2.883699,2.622699,2.322699,2.673699
c003,4470.611989,370.111989,10193.511990,38637.51199,5802.911989,2941.711989,5802.511989,1467.411989,835.011989,1050.011989,17577.51199,3338.211989,1318.311989,754.911989,702.611989,3444.511989,4097.511989,157.611989,14706.71199,42082.01199,5799.311989,3225.411989,4863.311989,1402.311989,3645.711989,1347.911989,588.011989,3924.011989,1067.911989,450.011989,492.411989,476.011989,888.61

In [ ]:
# SNIPPET 19
targets_df = targets_df.map({'hc': 0, 'sz': 1})
targets = targets_df.values.astype('int')

features = features_df.values.astype('float32')

## Feature engineering


### Feature extraction
In our example, we want to use neuroanatomical data to classify SZ and HC. This requires the extraction of brain morphometric information from the raw MRI images.


### Feature scaling/normalization 
In brain disorders research, we often deal with datasets that contain features that vary in units and range. However, to model the data correctly and effectively, most machine learning algorithms require the data to be on the same scale. Since normalization involves statistics (e.g. mean and variance) of the set used to train the model, in this point we split the data into training and test sets following the scheme a cross-validation.

In [ ]:
# SNIPPET 20
n_folds = 10
skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_seed)

![alt text](https://raw.githubusercontent.com/MLMH-Lab/How-To-Build-A-Machine-Learning-Model/master/figures/Figure%202.png)

In [ ]:
# SNIPPET 21
predictions_df = pd.DataFrame(targets_df)
predictions_df['predictions'] = np.nan

bac_cv = np.zeros((n_folds, 1))
sens_cv = np.zeros((n_folds, 1))
spec_cv = np.zeros((n_folds, 1))
coef_cv = np.zeros((n_folds, len(features_names)))

In [ ]:
# SNIPPET 22 REDO WITHOUT CV?!
for i_fold, (train_idx, test_idx) in enumerate(skf.split(features, targets)):
    features_train, features_test = features[train_idx], features[test_idx]
    targets_train, targets_test = targets[train_idx], targets[test_idx]

    print('CV iteration: %d' % (i_fold + 1))
    print('Training set size: %d' % len(targets_train))
    print('Test set size: %d' % len(targets_test))

    # --------------------------------------------------------------------------
    # SNIPPET 23
    # Feature scaling/normalization
    scaler = StandardScaler()

    scaler.fit(features_train)

    features_train_norm = scaler.transform(features_train)
    features_test_norm = scaler.transform(features_test)

    # --------------------------------------------------------------------------
    # SNIPPET 24
    # Here, we will use the linear kernel, as this will make it easier to extract the coefficients
    #  of the SVM model (feature importance) later on.
    clf = LinearSVC(loss='hinge')

    # --------------------------------------------------------------------------
    # SNIPPET 25
    # SVM relies on a hyperparameter C that regulates how much we want to avoid misclassifying each
    #  training example.

    # Hyper-parameter search space
    param_grid = {'C': [2 ** -6, 2 ** -5, 2 ** -4, 2 ** -3, 2 ** -2, 2 ** -1, 2 ** 0, 2 ** 1]}

    # Grid search
    internal_cv = StratifiedKFold(n_splits=10)
    grid_cv = GridSearchCV(estimator=clf,
                           param_grid=param_grid,
                           cv=internal_cv,
                           scoring='balanced_accuracy',
                           verbose=1)

    # --------------------------------------------------------------------------
    # SNIPPET 26
    # Model training
    grid_result = grid_cv.fit(features_train_norm, targets_train)

    # --------------------------------------------------------------------------
    # SNIPPET 27
    print('Best: %f using %s' % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']

    for mean, stdev, param in zip(means, stds, params):
        print('%f (%f) with: %r' % (mean, stdev, param))

    # --------------------------------------------------------------------------
    # SNIPPET 28
    best_clf = grid_cv.best_estimator_

    # --------------------------------------------------------------------------
    # SNIPPET 30
    # Model evaluation
    # Finally, we use the final trained model best_clf to make predictions in the test set.
    target_test_predicted = best_clf.predict(features_test_norm)

    # --------------------------------------------------------------------------
    # SNIPPET 31
    print('Confusion matrix')
    cm = confusion_matrix(targets_test, target_test_predicted)
    print(cm)

    tn, fp, fn, tp = cm.ravel()

    bac_test = balanced_accuracy_score(targets_test, target_test_predicted)
    sens_test = tp / (tp + fn)
    spec_test = tn / (tn + fp)

    print('Balanced accuracy: %.3f ' % bac_test)
    print('Sensitivity: %.3f ' % sens_test)
    print('Specificity: %.3f ' % spec_test)

    bac_cv[i_fold, :] = bac_test
    sens_cv[i_fold, :] = sens_test
    spec_cv[i_fold, :] = spec_test
    print('--------------------------------------------------------------------------')

CV iteration: 1
Training set size: 666
Test set size: 74
Fitting 10 folds for each of 8 candidates, totalling 80 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    7.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Best: 0.681996 using {'C': 0.03125}
0.656239 (0.082053) with: {'C': 0.015625}
0.681996 (0.084610) with: {'C': 0.03125}
0.672950 (0.086047) with: {'C': 0.0625}
0.681863 (0.084353) with: {'C': 0.125}
0.660561 (0.069703) with: {'C': 0.25}
0.663592 (0.070552) with: {'C': 0.5}
0.660517 (0.063467) with: {'C': 1}
0.650000 (0.070569) with: {'C': 2}
Confusion matrix
[[29  8]
 [13 24]]
Balanced accuracy: 0.716 
Sensitivity: 0.649 
Specificity: 0.784 
--------------------------------------------------------------------------
CV iteration: 2
Training set size: 666
Test set size: 74
Fitting 10 folds for each of 8 candidates, totalling 80 fits


[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    7.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Best: 0.684537 using {'C': 0.25}
0.657442 (0.060636) with: {'C': 0.015625}
0.675535 (0.071928) with: {'C': 0.03125}
0.683200 (0.069288) with: {'C': 0.0625}
0.683066 (0.061052) with: {'C': 0.125}
0.684537 (0.068300) with: {'C': 0.25}
0.678565 (0.069170) with: {'C': 0.5}
0.655971 (0.067399) with: {'C': 1}
0.650089 (0.080730) with: {'C': 2}
Confusion matrix
[[27 10]
 [10 27]]
Balanced accuracy: 0.730 
Sensitivity: 0.730 
Specificity: 0.730 
--------------------------------------------------------------------------
CV iteration: 3
Training set size: 666
Test set size: 74
Fitting 10 folds for each of 8 candidates, totalling 80 fits


...

[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    7.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Best: 0.672906 using {'C': 0.03125}
0.659492 (0.097342) with: {'C': 0.015625}
0.672906 (0.087826) with: {'C': 0.03125}
0.662389 (0.078545) with: {'C': 0.0625}
0.666934 (0.089875) with: {'C': 0.125}
0.657888 (0.086627) with: {'C': 0.25}
0.653298 (0.080025) with: {'C': 0.5}
0.654813 (0.077727) with: {'C': 1}
0.645900 (0.069074) with: {'C': 2}
Confusion matrix
[[30  7]
 [10 27]]
Balanced accuracy: 0.770 
Sensitivity: 0.730 
Specificity: 0.811 
--------------------------------------------------------------------------
CV iteration: 9
Training set size: 666
Test set size: 74
Fitting 10 folds for each of 8 candidates, totalling 80 fits


[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    7.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Best: 0.672193 using {'C': 0.0625}
0.648128 (0.076796) with: {'C': 0.015625}
0.652674 (0.078949) with: {'C': 0.03125}
0.672193 (0.068385) with: {'C': 0.0625}
0.651471 (0.077171) with: {'C': 0.125}
0.658913 (0.077373) with: {'C': 0.25}
0.640686 (0.073699) with: {'C': 0.5}
0.640686 (0.085128) with: {'C': 1}
0.642291 (0.081810) with: {'C': 2}
Confusion matrix
[[26 10]
 [ 9 29]]
Balanced accuracy: 0.743 
Sensitivity: 0.763 
Specificity: 0.722 
--------------------------------------------------------------------------
CV iteration: 10
Training set size: 666
Test set size: 74
Fitting 10 folds for each of 8 candidates, totalling 80 fits
Best: 0.681684 using {'C': 0.125}
0.661943 (0.088166) with: {'C': 0.015625}
0.664929 (0.102367) with: {'C': 0.03125}
0.676961 (0.082928) with: {'C': 0.0625}
0.681684 (0.070080) with: {'C': 0.125}
0.678788 (0.071619) with: {'C': 0.25}
0.660695 (0.059294) with: {'C': 0.5}
0.653119 (0.067356) with: {'C': 1}
0.657754 (0.081060) with: {'C': 2}
Confusion matrix
[[28

[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    7.2s finished


In [ ]:
# SNIPPET 32
print('CV results')
print('Bac: Mean(SD) = %.3f(%.3f)' % (bac_cv.mean(), bac_cv.std()))
print('Sens: Mean(SD) = %.3f(%.3f)' % (sens_cv.mean(), sens_cv.std()))
print('Spec: Mean(SD) = %.3f(%.3f)' % (spec_cv.mean(), spec_cv.std()))

CV results
Bac: Mean(SD) = 0.737(0.036)
Sens: Mean(SD) = 0.737(0.092)
Spec: Mean(SD) = 0.736(0.075)


## Post-hoc analysis

Once we have our final model, we can run several additional analyses. This tutotial does not include these analysis, but we could look at the following:

*   Test balanced accuracy, sensitivity and specificity for statistical significance via permutation testing
*   Identify the features that provided the greatest contribution to the task 